# Setup

Before starting the demo, install the python virtual environment and enable the use of this environemnt in jupyter.

Install and activate the python virtual environment:

    python3 -m venv .venv
    source .venv/bin/activate
    pip install -r requirements.txt

Enable jupyter to use the virtual environment, as nicely explained in this post: https://janakiev.com/blog/jupyter-virtual-envs/:

After having activated the virtual environment, run:
        
    pip install ipykernel
    python -m ipykernel install --user --name=.venv
    
Then start jupyter:

    jupyter notebook

## Example 1

Retrieve preprints that were posted within a given time range and that were published in journal from a given publisher.

In [ ]:
from src.biorxiv import retrieve

In [ ]:
prefix = '10.15252' # the doi prefix for EMBO Press
start = '2019-08-01' # start date of the time range
end = '2019-08-31' # end date of the time range

In [ ]:
results = retrieve(prefix, start, end)

In [ ]:
for preprint in results:
        print(preprint.biorxiv_doi, preprint.published_doi, preprint.preprint_category)

## Example 2

Posting a simple annotation and deleting it using the `toolbox` module.

To run this demonstration, you have to 

1. have a hypothes.is account and an api_key. See how to obtain an hypothes.is api key at https://web.hypothes.is/developers/.
2. create the group 'test' in your hypothes.is account, so that we can post a demonstration note to this group.
3. Edit `.env.example` to include your hypothes.is username and api_key and save this file as `.env`:
```
    # username for the hypothes.is account
    HYPOTHESIS_USER=...

    # hypothes.is api key associated with the user
    HYPOTHESIS_API_KEY=...
```

In [ ]:
from src import HYPO, HYPOTHESIS_USER # HYPO is an instance of hypothepy.v1.api.HypoApi that has been initialized with the crendentials of the user.
from src.utils import get_groupid # simple utility to retrieve the hyopthe.is groupid from the group name
from src.toolbox import HypoPost, Target, post_one # our tools to handle hypothes.is posts

In [ ]:
groupid = get_groupid('test') # retrieves the id assigned by hypothes.is to the group named test.
print(groupid)

Create the post using permissions that grants you update, delete and admin priviledge to the post and allows members of the private 'test' group to read it.

The post will be linked to the specified target page. It will have a smple '**Great work!**' markdown text and the tag 'EMBOpress'.

In [ ]:
perms = HYPO.helpers.permissions(
    read=[f'group:{groupid}'],
    update=[f'acct:{HYPOTHESIS_USER}@hypothes.is'],
    delete=[f'acct:{HYPOTHESIS_USER}@hypothes.is'],
    admin=[f'acct:{HYPOTHESIS_USER}@hypothes.is']
)
target = Target("https://www.embopress.org/journal/17444292", "Molecular Systems Biology")
post = HypoPost(annotation_text="**Great work!**", tags=['EMBOpress'])

Send the post to hypothes.is to add it to your group:

In [ ]:
response = post_one(perms, groupid, target, post)
if response.status_code == 200:
    hyp_id = response.json()['id']
    print(f"posted {hyp_id}")

Visit https://www.embopress.org/journal/17444292 open the hypothesis Chrome tool, login with your username and you should see the post under your private group.

You can also go to your hypothes.is account page and see the post there (use the groupid that was returned above): https://hypothes.is/groups/<groupid>/test

Now we cleanup and remove the post using the unique identifier assigned by hypothes.is upon posting:

In [ ]:
print(f"deleting {hyp_id}")
response = HYPO.annotations.delete(hyp_id)
if response.status_code == 200:
    print(f"deleted {hyp_id}")

Verify that the post has disappeared from https://hypothes.is/groups/<groupid>/test

## Example 3

Using a template to generate templated posts.

First, extend the base class `HypoPost` to add a method that generates the text of the annotation with a simple pyhon `string.Template` object.

In [ ]:
from string import Template
from src.toolbox import HypoPost

class HypoPostTemplated(HypoPost):
    """
    Extends HypoPost to generate a templated annotation.
    """

    def generate(self, msg: str, template: Template):
        """
        Generates the text of the post from a Template.
        
        Arguments:
            msg (str): the specific message to be inserted.
            template (Template): a simple string.Template object to generate the text of the annotation with appropriate substitution
        """

        self.annotation_text = template.substitute({'msg': msg})
        self.tags = ['EMBOPress']

Define the template with the appropriate `$msg` substitution variable:

In [ ]:
my_template = Template('**This is a simple demo of templated post.**\n$msg\n---\nIsn\'t it nice?') # this would be normally imported from a file

Create an instance of the HypoPostTemplated object and generate the markdoown text:

In [ ]:
my_post = HypoPostTemplated()
my_post.generate('### Great work!\n', my_template)
print(my_post.annotation_text)

Let's post it using the permissions `perms`, `groupid` and `target` from example 2 above.

In [ ]:
response = post_one(perms, groupid, target, my_post)
if response.status_code == 200:
    hyp_id = response.json()['id']
    print(f"posted {hyp_id}")

Check the nice post at https://www.embopress.org/journal/17444292 

And purge:

In [ ]:
print(f"deleting {hyp_id}")
response = HYPO.annotations.delete(hyp_id)
if response.status_code == 200:
    print(f"deleted {hyp_id}")